In [1]:
import pickle
from selenium import webdriver
import requests
import time
import bs4    
product_images = []




In [5]:


def get_cookies():
    browser.get("https://login.aliexpress.com/buyer.htm?return=https%3A%2F%2Fwww.aliexpress.com%2F&random=CEA73DF4D81D4775227F78080B9B6126")
    print('input your username and password in Firefox and hit Submit')
    input('Hit Enter here if you have summited the form: <Enter>')
    cookies = browser.get_cookies()
    pickle.dump(cookies, open("cookies.pickle", "wb"))


def set_cookies():
    browser.get("https://aliexpress.com")
    cookies = pickle.load(open("cookies.pickle", "rb"))
    for cookie in cookies:
        browser.add_cookie(cookie)
    browser.get("https://bestselling.aliexpress.com/en")

    

def get_item_info(product, sess, cookies):
    item_url = product['url']
    item_id = item_url.split('?')[0].split('/')[-1].split('.html')[0]
    
    
    print(item_url+'&isOrigTitle=true')

    r = sess.get(item_url+'&isOrigTitle=true', timeout=5.)
    item_soup = bs4.BeautifulSoup(r.text, 'lxml')
    title = item_soup.select_one('h1.product-name').text
    price = item_soup.select_one('span#j-sku-price').text

    try:
        price_disc = item_soup.select_one('span#j-sku-discount-price').text
    except:
        price_disc = None
        

    rating = float(item_soup.find(itemprop='ratingValue').text)
    review_num = int(item_soup.find(itemprop='reviewCount').text)
    order_num = int(''.join([x for x in item_soup.select_one('span#j-order-num').text if x.isdigit()]))
    
    print(review_num)

    props = {}
    for prop in item_soup.select('li.property-item'):
        props[prop.select_one('span.propery-title').text[:-1]] = prop.select_one('span.propery-des').text

    colors = []
    for prop in item_soup.select('dl.p-property-item'):
        if prop.select_one('dt.p-item-title').text == 'Color:':
            for color_item in prop.select('li'):                
                if color_item.select_one('img') is None:
                    color_url = ''
                else:
                    color_url = color_item.select_one('img').attrs['bigpic']
                colors.append({'color': color_item.select_one('a').attrs['title'],
                              'url': color_url})

        if prop.select_one('dt.p-item-title').text == 'Size:':
            sizes = [x.text for x in prop.select('li')]

    images = []
    for image_item in item_soup.select('span.img-thumb-item'):
        images.append(image_item.select_one('img').attrs['src'].replace('_50x50.jpg', '.jpg'))

    store = item_soup.select_one('a.store-lnk').text

    r = sess.get('http:'+item_soup.text.split('window.runParams.descUrl="')[-1].split('"')[0], timeout=5.)
    desc_soup = bs4.BeautifulSoup(r.text, 'lxml')
    desc_images = [x.attrs['src'] for x in desc_soup.select('img') if 'src' in x.attrs]
    feedback_url = 'http:'+item_soup.select_one('div#feedback').select_one('iframe').attrs['thesrc']

    feedbacks = []
    
    page = 1
    while True:
        r = sess.get(feedback_url+'&page=%d&withPictures=true'%page, timeout=10.)
        page += 1
        feedback_soup = bs4.BeautifulSoup(r.text, 'lxml')
        if len(feedback_soup.select('div.feedback-item')) == 0:
            break

        for feedback_item in feedback_soup.select('div.feedback-item'):
            user_country = feedback_item.select_one('div.user-country').text
            feedback_rating = int(feedback_item.select_one('span.star-view').select_one('span').attrs['style'].split('width:')[-1].split('%')[0])

            feedback_props_tmp = []
            for x in feedback_item.select_one('div.user-order-info').text.split('\n'):
                if x.strip() != '':
                    feedback_props_tmp.append(x.strip().replace(' ', ''))

            feedback_props = {}
            for i, x in enumerate(feedback_props_tmp):
                if i%2 == 0:
                    prop_name = x[:-1]
                else:
                    feedback_props[prop_name] = x
 
            try:
                feedback_assignment = feedback_item.select_one('span.product-feedback-info').text.strip()
            except:
                feedback_assignment = None

            personal_info = {}
            for personal_item in feedback_item.select('span.buyer-personal-info-item'):
                personal_item_title = personal_item.select_one('span.buyer-personal-info-title').text[:-1]
                personal_item_value = ''.join(personal_item.findAll(text=True, recursive=False)).strip()
                personal_info[personal_item_title] = personal_item_value

            feedback_images = [x.attrs['data-src'] for x in feedback_item.select('li.pic-view-item')]
            feedback_text = feedback_item.select_one('dt.buyer-feedback').text.strip()
            feedback_time = feedback_item.select_one('dd.r-time').text.strip()

            feedbacks.append({
                'country': user_country,
                'rating': feedback_rating,
                'props': feedback_props,
                'assign': feedback_assignment,
                'person': personal_info,
                'imgs': feedback_images,
                'txt': feedback_text,
                'time': feedback_time,
            })

    item_info = {
        'url': item_url,
        'id': item_id,
        'title': title,
        'price': price,
        'price_disc': price_disc,
        'rating': rating,
        'review_num': review_num,
        'order_num': order_num,
        'props': props,
        'colors': colors,
        'images': images,
        'store': store,
        'desc_images': desc_images,
        'feedbacks': feedbacks,
        'cat': product['cat'],
        'subcat': product['subcat'],
    }

    return item_info
                  
def extract_product_urls_from_list_page(list_page_url):
    print(list_page_url)
    browser.get(list_page_url)
    time.sleep(1)
    cats = browser.find_elements_by_css_selector('.picRind')
    print(cats)

    all_links = set()
    for ind, cat in enumerate(cats):
        try:
            print("---")
            print(cat)
            print(cat.size)
            link = (cat.get_attribute('href'))
            print(link)

            product = {'url':link, 'cat': cat, 'subcat': 'apparel'}    
            print(product)


            info = get_item_info(product, sess, cookies)    
            print("info:")
            print(info)
            print("*********************")

            feedbacks = info['feedbacks']
            images = info['images']
            desc_images = info['desc_images']
            
            print(desc_images)
            
            imgs = []
            for feedback_image in feedbacks:
                for image in feedback_image['imgs']:
                    imgs.append(image)
            for image in images:                
                    imgs.append(image)
            for image in desc_images:                
                    imgs.append(image)
                    
            print(imgs)

            product_images.append(imgs)


            all_links.add(link)
            time.sleep(0.1)
        except:
            pass
    return all_links

def get_categories(browser, root_link):
    browser.get(root_link)

    soup=bs4.BeautifulSoup(browser.page_source)

    #do something useful
    #prints all the links with corresponding text

    categories = []
    for link in soup.find_all('a'):
        try:
            if link.get('href',None).find("www.aliexpress.com/category/") > 0:
                print(link.get('href',None))
                #print(link.get('href',None),link.get_text())
                categories.append("https:"+link.get('href',None))
        except:
            pass
    print(categories)


    subcategories = []
    print("parsing subcategories:")
    for category in categories:
        print(category)
        browser.get(category)

        soup=bs4.BeautifulSoup(browser.page_source)

        #do something useful
        #prints all the links with corresponding text

        categories = []
        table = soup.find("dl", {"class": "category-list"})
        #print(table.text)



        for link in table.findChildren("a" , recursive=True):

            #print("this is link:")

            #print(link.get('href',None))
            #print(link.getText())
            try:
                if link.get('href',None).find("www.aliexpress.com/category/") > 0:
                    #print(link.get('href',None))
                    #print(link.get('href',None),link.get_text())
                    subcategories.append("https:"+link.get('href',None))
            except:
                pass
    #print(subcategories)

    print(len(subcategories))

    for category in subcategories:
        print(category)





In [3]:
if __name__ == '__main__':
    browser = webdriver.Firefox()
    get_cookies()
    set_cookies()
   
    

input your username and password in Firefox and hit Submit
Hit Enter here if you have summited the form: <Enter>


In [4]:

sess = requests.Session()
cookies = pickle.load(open("cookies.pickle", "rb"))

links = []

#for page in range(1, 2):
#      links.append((extract_product_urls_from_list_page('https://www.aliexpress.com/category/200003478/trench.html?site=glo&g=y&SortType=total_tranpro_desc&needQuery=n&isrefine=y&page='+str(page))))


root_link = 'http://aliexpress.com'

categories = get_categories(browser, root_link)


"""        
with open('filtered_categories2.txt', 'r') as f:
    subcategories = f.readlines()
    
    for category in subcategories:
        print(category)        
        for page in range(1, 10):
            extract_product_urls_from_list_page(category+'&SortType=total_tranpro_desc&page='+str(page))        
"""            

//www.aliexpress.com/category/100003109/women-clothing-accessories.html
//www.aliexpress.com/category/100003070/men-clothing-accessories.html
//www.aliexpress.com/category/5090301/mobile-phones.html
//www.aliexpress.com/category/200084017/mobile-phone-accessories.html
//www.aliexpress.com/category/7/computer-office.html
//www.aliexpress.com/category/21/office-school-supplies.html
//www.aliexpress.com/category/30/security-protection.html
//www.aliexpress.com/category/44/consumer-electronics.html
//www.aliexpress.com/category/1509/jewelry-accessories.html
//www.aliexpress.com/category/1511/watches.html?isCates=y
//www.aliexpress.com/category/15/home-garden.html
//www.aliexpress.com/category/6/home-appliances.html
//www.aliexpress.com/category/1524/luggage-bags.html
//www.aliexpress.com/category/322/shoes.html
//www.aliexpress.com/category/26/toys-hobbies.html
//www.aliexpress.com/category/1501/mother-kids.html
//www.aliexpress.com/category/18/sports-entertainment.html
//www.aliexpress.co

"        \nwith open('filtered_categories2.txt', 'r') as f:\n    subcategories = f.readlines()\n    \n    for category in subcategories:\n        print(category)        \n        for page in range(1, 10):\n            extract_product_urls_from_list_page(category+'&SortType=total_tranpro_desc&page='+str(page))        \n"